In [1]:
import pandas as pd
import pickle as pkl
import datetime as dt
pd.set_option('display.max_columns',300)

In [2]:
df = pd.read_csv('data/full_alerts.csv')

In [3]:
df['message_lower'] = df['message'].str.lower()
df['subj_lower'] = df['subject'].str.lower()

In [4]:
s = pd.Series(df['date'])

In [5]:
s = pd.to_datetime(s)

In [6]:
df['day_of_week'] = s.dt.dayofweek

In [7]:
df['date'][5][-5:]

'21:28'

In [8]:
def rush_hour(time):
    if ((time[-5:] >= '07:00' and time[-5:] <= '09:00') or (time[-5:] >= '15:00' and time[-5:] <= '18:30')):
        return 1
    else:
        return 0
def rush_hour_day(day):
    if (day >= 0 and day <= 4):
        return 1
    else:
        return 0

def weekend(day):
    if (day == 5 or day == 6):
        return 1
    else:
        return 0

In [9]:
df['rush_hour'] = (df['date'].apply(rush_hour) & df['day_of_week'].apply(rush_hour_day))

In [10]:
df['weekend'] = df['day_of_week'].apply(weekend)

In [11]:
df['date'] = pd.to_datetime(df['date'])

In [12]:
def get_one(value):
    if value is True:
        return 1
    else:
        return 0

In [13]:
df['bronx'] = df['subject'].str.contains('bx')
df['manhattan'] = df['subject'].str.contains('manh')
df['brooklyn'] = df['subject'].str.contains('bk')
df['queens'] = df['subject'].str.contains('qns')

df['brooklyn'] = df['brooklyn'].apply(get_one)
df['queens'] = df['queens'].apply(get_one)
df['bronx'] = df['bronx'].apply(get_one)
df['manhattan'] = df['manhattan'].apply(get_one)

In [14]:
nyc = pd.read_csv('data_cleaning/nyc_weather.csv')

In [15]:
nyc['date'] = pd.to_datetime(nyc['date'])

In [16]:
nyc.set_index('date',inplace=True)

In [17]:
df['dayz'] = df['date'].dt.date

In [18]:
nyc['datetime'] = pd.to_datetime(nyc.index)
nyc['dayz'] = nyc['datetime'].dt.date

In [19]:
df = df.merge(nyc, how='left', left_on=df['dayz'], right_on=nyc['dayz'])

In [20]:
df.drop(['key_0','datetime','dayz_x', 'dayz_y'],axis=1,inplace=True)

In [21]:
df.rename(columns={0:'mon',1:'tue',2:'wed',3:'th',4:'fri',5:'sat',6:'sun'},inplace=True)

In [22]:
days = pd.get_dummies(df['day_of_week'])

In [23]:
days.rename(columns={0:'mon',1:'tue',2:'wed',3:'th',4:'fri',5:'sat',6:'sun'},inplace=True)
df = df.merge(days,left_index=True,right_index=True)
df.drop('day_of_week',axis=1,inplace=True)
df

,date,subject,message,message_lower,subj_lower,rush_hour,weekend,bronx,manhattan,brooklyn,queens,avg_temp,min_temp,max_temp,snowfall,snowfall_depth,prcp,avg_wind_spd,fastest_wind_directions,fastest_wind_spd,major_conditions,minor_conditions,neutral_conditions,clear_conditions,mon,tue,wed,th,fri,sat,sun
0,2021-09-30 23:54:00,"bk , Franklin Av S Train , Some Delays",Franklin Av Shuttle trains are delayed in both...,franklin av shuttle trains are delayed in both...,"bk , franklin av s train , some delays",0,0,0,0,1,0,62.0,54.5,66.0,0.0,0.0,0.000,9.955,"nw, n",26.95,0.0,0.0,0.0,1.0,0,0,0,1,0,0,0
1,2021-09-30 23:38:00,"bk , Q Train , Delays",Q trains are delayed in both directions while ...,q trains are delayed in both directions while ...,"bk , q train , delays",0,0,0,0,1,0,62.0,54.5,66.0,0.0,0.0,0.000,9.955,"nw, n",26.95,0.0,0.0,0.0,1.0,0,0,0,1,0,0,0
2,2021-09-30 22:14:00,"manh , L Train , Delays",L trains are delayed entering/leaving 8 Av whi...,l trains are delayed entering/leaving 8 av whi...,"manh , l train , delays",0,0,0,1,0,0,62.0,54.5,66.0,0.0,0.0,0.000,9.955,"nw, n",26.95,0.0,0.0,0.0,1.0,0,0,0,1,0,0,0
3,2021-09-30 22:13:00,"bx , 4 Train , Delays",4 trains are delayed entering/leaving Woodlawn...,4 trains are delayed entering/leaving woodlawn...,"bx , 4 train , delays",0,0,1,0,0,0,62.0,54.5,66.0,0.0,0.0,0.000,9.955,"nw, n",26.95,0.0,0.0,0.0,1.0,0,0,0,1,0,0,0
4,2021-09-30 21:34:00,"bk , F and G Trains , Delays",Queens-bound F G trains are delayed while we g...,queens-bound f g trains are delayed while we g...,"bk , f and g trains , delays",0,0,0,0,1,0,62.0,54.5,66.0,0.0,0.0,0.000,9.955,"nw, n",26.95,0.0,0.0,0.0,1.0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83132,2010-01-01 09:50:00,"manh , 1 train , Police Investigation","South Ferry-bound 1 trains , run express , b...","south ferry-bound 1 trains , run express , b...","manh , 1 train , police investigation",0,0,0,1,0,0,36.0,32.5,40.5,0.0,0.0,0.035,4.365,"nw, w",22.45,1.0,0.0,0.0,0.0,0,0,0,0,1,0,0
83133,2010-01-01 09:10:00,"manh , C train , Mechanical Problems","168th St-bound C trains , run express , btwn...","168th st-bound c trains , run express , btwn...","manh , c train , mechanical problems",0,0,0,1,0,0,36.0,32.5,40.5,0.0,0.0,0.035,4.365,"nw, w",22.45,1.0,0.0,0.0,0.0,0,0,0,0,1,0,0
83134,2010-01-01 01:57:00,"manh , 1 Trains , Signal Problems",Bx-bound . 1 train running w/delays . Due to...,bx-bound . 1 train running w/delays . due to...,"manh , 1 trains , signal problems",0,0,0,1,0,0,36.0,32.5,40.5,0.0,0.0,0.035,4.365,"nw, w",22.45,1.0,0.0,0.0,0.0,0,0,0,0,1,0,0
83135,2010-01-01 01:15:00,"bk , J Train , Police Investigation",Both directions . J train susp Myrtle Av to C...,both directions . j train susp myrtle av to c...,"bk , j train , police investigation",0,0,0,0,1,0,36.0,32.5,40.5,0.0,0.0,0.035,4.365,"nw, w",22.45,1.0,0.0,0.0,0.0,0,0,0,0,1,0,0


In [24]:
def become_one(one):
    if one >= 1:
        return 1
    else:
        return 0

def become_zero_and_one(number):
    if number == 0:
        return 1
    if number == 1:
        return 0

In [25]:
delayed_1 = df.loc[df['subj_lower'].str.contains(' 1 ')]
delayed_2 = df.loc[df['subj_lower'].str.contains(' 2 ')]
delayed_3 = df.loc[df['subj_lower'].str.contains(' 3 ')]
delayed_4 = df.loc[df['subj_lower'].str.contains(' 4 ')]
delayed_5 = df.loc[df['subj_lower'].str.contains(' 5 ')]
delayed_6 = df.loc[df['subj_lower'].str.contains(' 6 ')]
delayed_7 = df.loc[df['subj_lower'].str.contains(' 7 ')]
delayed_s = df.loc[df['subj_lower'].str.contains('42 st sh')]

delayed_1_ind = delayed_1.index
delayed_2_ind = delayed_2.index
delayed_3_ind = delayed_3.index
delayed_4_ind = delayed_4.index
delayed_5_ind = delayed_5.index
delayed_6_ind = delayed_6.index
delayed_7_ind = delayed_7.index
delayed_s_ind = delayed_s.index

In [26]:
df.insert(0, 'delayed_1', 0)
df.insert(0, 'delayed_2', 0)
df.insert(0, 'delayed_3', 0)
df.insert(0, 'delayed_4', 0)
df.insert(0, 'delayed_5', 0)
df.insert(0, 'delayed_6', 0)
df.insert(0, 'delayed_7', 0)
df.insert(0, 'delayed_42_st_S', 0)
df.insert(0, 'delayed_irt', 0)

In [27]:
df.loc[delayed_1_ind, 'delayed_1'] = 1
df.loc[delayed_2_ind, 'delayed_2'] = 1
df.loc[delayed_3_ind, 'delayed_3'] = 1
df.loc[delayed_4_ind, 'delayed_4'] = 1
df.loc[delayed_5_ind, 'delayed_5'] = 1
df.loc[delayed_6_ind, 'delayed_6'] = 1
df.loc[delayed_7_ind, 'delayed_7'] = 1
df.loc[delayed_s_ind, 'delayed_42_st_S'] = 1

In [28]:
df['delayed_irt'] = (df['delayed_1'] + df['delayed_2'] + df['delayed_3'] + df['delayed_4'] + df['delayed_5']
                     + df['delayed_6'] + df['delayed_7'] + df['delayed_42_st_S'])
df['delayed_irt'] = df['delayed_irt'].apply(become_one)

In [29]:
causes_list = ['track', 'customer','passenger','mechanical','maintenance','signal','switch','brake','struck', 'light', 'station',
          'weather','door','police','nypd','fire','fdny','emt','ems','investigation','remov','smoke','medical', 'flood','water',
         'construction','reconstruction','repairs','assault','surf','accident','storm','winter','debris','malfunction',
         'cold','power', 'forecast','rail']
direction_list = ['northbound','soundbound','both direction','nb','sb','bd','bothdirections','both way','bothway','n /','n/b',
            'n/ b','s /','s/b','s/ b','b/d','b / d','n / b','s / b','b/ d','b /d','b/ ','manhattan bound', 'brooklyn bound',
            'queens bound', 'bronx bound','manh bound','bk bound','bx bound','qns bound']

In [30]:
df['causes'] = df['message_lower'].str.extract('({0})'.format('|'.join(causes_list)))

In [31]:
causes = pd.get_dummies(df['causes'])
causes

,accident,assault,brake,cold,construction,customer,debris,door,ems,fdny,fire,flood,forecast,investigation,light,maintenance,malfunction,mechanical,medical,nypd,passenger,police,power,rail,reconstruction,remov,repairs,signal,smoke,station,storm,struck,surf,switch,track,water,weather,winter
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83132,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
83133,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
83134,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
83135,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
# unruly passengers or medical emergency and such
causes['causes_passenger'] = (causes['nypd'] + causes['investigation'] + causes['police'] + causes['ems']
                      + causes['medical'] + causes['passenger'] + causes['customer'] + causes['struck'] + causes['assault']
                             + causes['surf'])
causes['causes_passenger'] = causes['causes_passenger'].apply(become_one)

# issues at the station including switches, signals, tracks, people/trains/debris needing to be removed from the track
causes['causes_station'] = (causes['signal'] + causes['switch'] + causes['track'] + causes['remov'] + causes['maintenance']
                           + causes['station'] + causes['fire'] + causes['smoke'] + causes['flood'] + causes['water'] + causes['malfunction']
                           + causes['power'] + causes['construction'] + causes['debris'] + causes['reconstruction']
                           + causes['rail'] + causes['fdny'])
causes['causes_station'] = causes['causes_station'].apply(become_one)

causes['causes_train'] = (causes['brake'] + causes['light'] + causes['door'] + causes['mechanical'] + causes['accident']
                         + causes['malfunction'] + causes['repairs'] + causes['weather'])
causes['causes_train'] = causes['causes_train'].apply(become_one)

causes['causes_weather'] = (causes['storm'] + causes['cold'] + causes['forecast'] + causes['winter'])
causes['causes_weather'] = causes['causes_weather'].apply(become_one)

In [33]:
causes.drop(['nypd','investigation','police','ems','medical','passenger','customer','struck','assault',
            'signal','switch','track','remov','maintenance','station','fire','smoke','weather','flood',
            'water','brake','light','door','mechanical','accident','malfunction','surf','construction','power',
            'debris','winter','storm','repairs','cold','reconstruction', 'forecast','rail','fdny'],axis=1,inplace=True)

In [34]:
causes['causes_other'] = causes['causes_passenger'] + causes['causes_station'] + causes['causes_train'] + causes['causes_weather']
causes['causes_other'] = causes['causes_other'].apply(become_zero_and_one)
causes['causes_other'] = causes['causes_other'].apply(become_one)

In [35]:
causes

,causes_passenger,causes_station,causes_train,causes_weather,causes_other
0,0,1,0,0,0
1,0,1,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
83132,1,0,0,0,0
83133,0,0,1,0,0
83134,0,1,0,0,0
83135,1,0,0,0,0


In [36]:
df['direction_msg'] = df['message_lower'].str.extract('({0})'.format('|'.join(direction_list)))

df['direction_sub'] = df['subj_lower'].str.extract('({0})'.format('|'.join(direction_list)))

In [37]:
direction_1 = pd.get_dummies(df['direction_msg'])
direction_2 = pd.get_dummies(df['direction_sub'])

direction = direction_1.merge(direction_2,left_index=True,right_index=True)

In [38]:
print(list(direction.columns))

['b/ ', 'b/ d', 'bd', 'both direction_x', 'bronx bound', 'brooklyn bound_x', 'manhattan bound', 'n /_x', 'n/ b', 'n/b', 'nb', 'northbound_x', 'qns bound_x', 'queens bound_x', 's /_x', 's/ b', 's/b', 'sb_x', 'bk bound', 'both direction_y', 'brooklyn bound_y', 'bx bound', 'manh bound', 'n /_y', 'northbound_y', 'qns bound_y', 'queens bound_y', 's /_y', 'sb_y']


In [39]:
direction['direction_bronx'] = (direction['nb'] + direction['bx bound']
                                + direction['n /_x'] + direction['n /_y'] + direction['northbound_x'] + direction['n/b']
                                + direction['northbound_y'] + direction['bronx bound'] + direction['n/ b'])
direction['direction_bronx'] = direction['direction_bronx'].apply(become_one)

direction['direction_brooklyn'] = (direction['bk bound'] + direction['s /_y'] + direction['s /_x']
                                   + direction['sb_x'] +  direction['sb_y'] + direction['s/ b'] + 
                                   direction['brooklyn bound_x'] + direction['brooklyn bound_y'] + direction['s/b']
                                  + direction['bk bound'])
direction['direction_brooklyn'] = direction['direction_brooklyn'].apply(become_one)

direction['direction_manhattan'] = (direction['manh bound'] + direction['manhattan bound'])
direction['direction_manhattan'] = direction['direction_manhattan'].apply(become_one)

direction['direction_queens'] = (direction['qns bound_x'] + direction['qns bound_y'] + direction['queens bound_x']
                                 + direction['queens bound_y'])
direction['direction_queens'] = direction['direction_queens'].apply(become_one)

# both/unspec directions
direction['direction_unspecified'] = (direction['b/ '] + direction['b/ d'] + direction['both direction_x']
                               + direction['bd'] + direction['both direction_y'])
direction['direction_unspecified'] = direction['direction_unspecified'].apply(become_one)

In [40]:
direction.drop(['b/ ', 'b/ d', 'bd', 'both direction_x', 'bronx bound', 'brooklyn bound_x', 'manhattan bound',
                'n /_x', 'n/ b', 'n/b', 'nb', 'northbound_x', 'qns bound_x', 'queens bound_x', 's /_x', 's/ b', 's/b',
                'sb_x', 'bk bound', 'both direction_y', 'brooklyn bound_y','bx bound', 'manh bound', 'n /_y', 'northbound_y',
                'qns bound_y', 'queens bound_y', 's /_y', 'sb_y'],axis=1,inplace=True)

In [41]:
df = df.merge(causes, how='left', left_index=True,right_index=True)
df = df.merge(direction, how='left', left_index=True,right_index=True)
df.drop(['causes','direction_msg','direction_sub'],axis=1,inplace=True)

In [42]:
df['major_conditions'] = df['major_conditions'].apply(become_one)
df['minor_conditions'] = df['minor_conditions'].apply(become_one)
df['neutral_conditions'] = df['neutral_conditions'].apply(become_one)
df['clear_conditions'] = df['clear_conditions'].apply(become_one)

In [43]:
df

,delayed_irt,delayed_42_st_S,delayed_7,delayed_6,delayed_5,delayed_4,delayed_3,delayed_2,delayed_1,date,subject,message,message_lower,subj_lower,rush_hour,weekend,bronx,manhattan,brooklyn,queens,avg_temp,min_temp,max_temp,snowfall,snowfall_depth,prcp,avg_wind_spd,fastest_wind_directions,fastest_wind_spd,major_conditions,minor_conditions,neutral_conditions,clear_conditions,mon,tue,wed,th,fri,sat,sun,causes_passenger,causes_station,causes_train,causes_weather,causes_other,direction_bronx,direction_brooklyn,direction_manhattan,direction_queens,direction_unspecified
0,0,0,0,0,0,0,0,0,0,2021-09-30 23:54:00,"bk , Franklin Av S Train , Some Delays",Franklin Av Shuttle trains are delayed in both...,franklin av shuttle trains are delayed in both...,"bk , franklin av s train , some delays",0,0,0,0,1,0,62.0,54.5,66.0,0.0,0.0,0.000,9.955,"nw, n",26.95,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,2021-09-30 23:38:00,"bk , Q Train , Delays",Q trains are delayed in both directions while ...,q trains are delayed in both directions while ...,"bk , q train , delays",0,0,0,0,1,0,62.0,54.5,66.0,0.0,0.0,0.000,9.955,"nw, n",26.95,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,2021-09-30 22:14:00,"manh , L Train , Delays",L trains are delayed entering/leaving 8 Av whi...,l trains are delayed entering/leaving 8 av whi...,"manh , l train , delays",0,0,0,1,0,0,62.0,54.5,66.0,0.0,0.0,0.000,9.955,"nw, n",26.95,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,2021-09-30 22:13:00,"bx , 4 Train , Delays",4 trains are delayed entering/leaving Woodlawn...,4 trains are delayed entering/leaving woodlawn...,"bx , 4 train , delays",0,0,1,0,0,0,62.0,54.5,66.0,0.0,0.0,0.000,9.955,"nw, n",26.95,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,2021-09-30 21:34:00,"bk , F and G Trains , Delays",Queens-bound F G trains are delayed while we g...,queens-bound f g trains are delayed while we g...,"bk , f and g trains , delays",0,0,0,0,1,0,62.0,54.5,66.0,0.0,0.0,0.000,9.955,"nw, n",26.95,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83132,1,0,0,0,0,0,0,0,1,2010-01-01 09:50:00,"manh , 1 train , Police Investigation","South Ferry-bound 1 trains , run express , b...","south ferry-bound 1 trains , run express , b...","manh , 1 train , police investigation",0,0,0,1,0,0,36.0,32.5,40.5,0.0,0.0,0.035,4.365,"nw, w",22.45,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
83133,0,0,0,0,0,0,0,0,0,2010-01-01 09:10:00,"manh , C train , Mechanical Problems","168th St-bound C trains , run express , btwn...","168th st-bound c trains , run express , btwn...","manh , c train , mechanical problems",0,0,0,1,0,0,36.0,32.5,40.5,0.0,0.0,0.035,4.365,"nw, w",22.45,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
83134,1,0,0,0,0,0,0,0,1,2010-01-01 01:57:00,"manh , 1 Trains , Signal Problems",Bx-bound . 1 train running w/delays . Due to...,bx-bound . 1 train running w/delays . due to...,"manh , 1 trains , signal problems",0,0,0,1,0,0,36.0,32.5,40.5,0.0,0.0,0.035,4.365,"nw, w",22.45,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
83135,0,0,0,0,0,0,0,0,0,2010-01-01 01:15:00,"bk , J Train , Police Investigation",Both directions . J train susp Myrtle Av to C...,both directions . j train susp myrtle av to c...,"bk , j train , police investigation",0,0,0,0,1,0,36.0,32.5,40.5,0.0,0.0,0.035,4.365,"nw, w",22.45,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1


In [44]:
with open('data/new_hope.pkl','wb') as f:
    pkl.dump(df, f)